Counter mode only provides confidentiality and changing one byte by an attacker only changes one byte of plaintext, unlike Cipher block chaining mode

In [1]:
# This demonstrates how someone could change a message if the knew the plain text and position, but not the key
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

# Key and nonce (IV for CTR mode) generation
key = os.urandom(32)
nonce = os.urandom(16)

# Create a plaintext message
message = "This is a secret message. Don't let anyone see it."

# Create a cipher object
aesCipher = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend())

# Encryption
aesEncryptor = aesCipher.encryptor()
ciphertext = aesEncryptor.update(message.encode()) + aesEncryptor.finalize()

# At this point, suppose an attacker knows that "Don't let anyone see it." is part of the plaintext.
# They want to change that to "Attack at dawn."

known_plaintext = "Don't let anyone see it.".encode()
new_plaintext =   "Attack at dawn.".encode()
known_ciphertext = ciphertext[-len(known_plaintext):]  # Assuming the attacker knows where in the ciphertext this part is

# The zip() function in Python returns an iterator of tuples, where the i-th tuple contains the i-th element from each
# of the input iterables. The iterator stops when the shortest input iterable is exhausted.

# So if encrypted_counter and new_plaintext are not of the same length, the zip() function will stop at the end
# of the shortest one. This means the new_ciphertext will be truncated to the length of the shortest input iterable.

# This scenario could happen if the new_plaintext you're trying to substitute is longer than the known_plaintext.
# In the provided code example, it's assumed that the new_plaintext and the known_plaintext are of the same length
# to avoid such a situation.

# In a more generalized scenario where you're trying to substitute a new_plaintext that's longer than the known_plaintext,
# you'll face problems because AES-CTR uses a unique counter value for each block of plaintext. If your new plaintext 
# spans more blocks than the known plaintext, you won't have the correct counter values for the additional blocks. To 
# successfully substitute a longer plaintext, you'd need to know the entire plaintext and ciphertext, or at least 
# enough of it to determine the counter values for all blocks you want to modify.

# The attacker can XOR the known ciphertext with the known plaintext to recover the encrypted counter
encrypted_counter = bytes(a ^ b for a, b in zip(known_ciphertext, known_plaintext))

# Then XOR that with the new plaintext to produce a new ciphertext
new_ciphertext = bytes(a ^ b for a, b in zip(encrypted_counter, new_plaintext))

# The attacker replaces the old ciphertext with the new one in the message
modified_ciphertext = ciphertext[:-len(known_plaintext)] + new_ciphertext

# Decryption
aesDecryptor = aesCipher.decryptor()
decryptedtext = aesDecryptor.update(modified_ciphertext) + aesDecryptor.finalize()

print("Decrypted message: ", decryptedtext.decode())

Decrypted message:  This is a secret message. Attack at dawn.


In [1]:
# This demonstrates how someone could determine where multiple messages are the same if the same IV and key is used for them. 
# This allows an attacker to know what part of the plaintext to focus on capturing so they could modify future messages or get the keys
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

# Key and nonce (IV for CTR mode) generation
key = os.urandom(32)
nonce = os.urandom(16)

# Create a plaintext message
message1 = "This is the header.ABCDEFGHIJKLMNOPQRSTUVWXYZ"
message2 = "This is the header.BCDEFGHIJKLMNOPQRSTUVWXYZA"

# Create a cipher object
aesCipher1 = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend())
aesCipher2 = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend())

# Encryption
aesEncryptor1 = aesCipher1.encryptor()
ciphertext1 = aesEncryptor1.update(message1.encode()) + aesEncryptor1.finalize()

aesEncryptor2 = aesCipher2.encryptor()
ciphertext2 = aesEncryptor2.update(message2.encode()) + aesEncryptor2.finalize()

# The attacker can look for where the message is unchanged this is because
# c1 = m1 ^ K
# c2 = m2 ^ K
# c1 ^ c2 = (m1 ^ K) ^ (m2 ^ K)
# c1 ^ c2 = (m1 ^ m2) ^ (K ^ K)
# c1 ^ c2 = (m1 ^ m2) # This will be zero where the message is the same
c1_xor_c2_message = bytes(a ^ b for a, b in zip(ciphertext1, ciphertext2))

is_fixed_text = [ 'Y' if c1_xor_c2 == 0 else '.'  for c1_xor_c2 in c1_xor_c2_message]
print(f"is_fixed_text: {''.join(is_fixed_text)}")
print(f"    message 1: {message1}")
print(f"    message 2: {message2}")

is_fixed_text: YYYYYYYYYYYYYYYYYYY..........................
    message 1: This is the header.ABCDEFGHIJKLMNOPQRSTUVWXYZ
    message 2: This is the header.BCDEFGHIJKLMNOPQRSTUVWXYZA
